# Materials

In PyMoosh, the part dealing with materials is a bit specific, to manage all the data necessary for the computations. 

Here we show a little bit how this works and what you can do with it. 

In order to make things simple for the user, when a Structure is defined the informations which are given are used to generate an object which belongs to the `Material` class.

In [13]:
from PyMoosh import *

air = Material(1.)

`air`is thus a material, which means you can access its permittivity and permeability easily...

In [14]:
# Wavelength in nanometer
wavelength = 600
epsilon = air.get_permittivity(wavelength)
mu = air.get_permeability(wavelength)
print("epsilon = ",epsilon,"\nmu =",mu)

epsilon =  (1+0j) 
mu = 1.0


This was easy. Now we can declare another material but may want to specify its permeability, because it is different from one. It is still a non-dispersive material which we could call `metamaterial` for instance. It has a magnetic response, but it is not dispersive.

To do so, simply give two values to `Material` : `(epsilon, mu)`

In [15]:
metamaterial = Material([-1.,-2.])
epsilon = metamaterial.get_permittivity(wavelength)
mu = metamaterial.get_permeability(wavelength)
print("epsilon = ",epsilon,"\nmu =",mu)

epsilon =  -1.0 
mu = -2.0


Now, you may want to be able to define a material, like an ideal metal which has a permittivity described by a simple Drude model. What is important, is that the permittivity can be defined by a function that only depends on the wavelength.

In [16]:
def drude(wavelength):
    return 1 - wavelength**2 / 125 **2

And then define a material by giving a function as an argument to `Material`. You can check that in this case the material is dispersive. 

In [17]:
metal = Material(drude)
wavelength = 400
epsilon = metal.get_permittivity(wavelength)
mu = metal.get_permeability(wavelength)
print("epsilon = ",epsilon,"\nmu =",mu)

epsilon =  -9.24 
mu = 1.0


These types, in addition to the simple access to the local database, are the default types for the Material class.

**Any of those materials can be provided to declare a class `Structure`, it will be used as is.**

# Models

There are several cases where you want to have a more complex function to define your material's permittivity.

Many typical such functions (usual models like Drude or Brendel&Bormann, but also experimental data interpolation) can be found in the `model` module.

The difference with the simple `drude` functions used above, is now that these functions can take extra parameters.

The syntax now is `Material([function, params], specialType="Model")`

In [18]:
import numpy as np 
wavelength_list = np.linspace(300,800,100)
# Let us use our Drude model declared above to generate the corresponding permittivities :
permittivities = drude(wavelength_list)

from PyMoosh.models import ExpData
# You can import it directly if you know how it works, or copy/paste the function to your code

metal2 = Material([ExpData, wavelength_list, permittivities], specialType="Model")

wavelength  = 500
epsilon = metal2.get_permittivity(wavelength)
mu = metal2.get_permeability(wavelength)
print("The interpolation provides : \nepsilon = ",epsilon,"\nmu =",mu)
print("Drude model provides : epsilon =",metal.get_permittivity(500))

The interpolation provides : 
epsilon =  -15.000391796755434 
mu = 1.0
Drude model provides : epsilon = -15.0


As you can see, the values are not exactly the same. First for experimental data, it is always assumed epsilon is complex. Then, it is an interpolation between experimental points, so this is not as accurate as an analytic formula.

# Refractive Index Database integration

Now, one massively used source of permittivity data is the RefractiveIndex.info database. PyMoosh can interact with it now!

Materials in the RII database are stored on shelves (general type of material), then books (material), then page (source of refractive index for this material (experiment/model...).)

So the syntax is now `Material(["shelf", "book", "page"], specialType="RII")`

In [20]:
# Example for SiO2

shelf = "main"
book = "SiO2"
page = "Nyakuchena" #latest to date

silica = Material([shelf, book, page], specialType="RII")
epsilon = silica.get_permittivity(1500)
# Important: The RII database will give off errors if the wavelength is not within
# the stored range (no extrapolation is allowed)
print("epsilon = ",epsilon)


epsilon =  2.0186092372978477
